In [2]:
from Fitting import Fitting
import json

In [135]:
with open("ASHRAE_CR3-1.json") as file:
    elbow = json.load(file)

In [136]:
fit1 = Fitting(elbow)

In [137]:
fit1.fittype

'ELBOW'

In [138]:
mycondition={}
mycondition["H_i"] = 200
mycondition["W_i"] = 200
mycondition["r"] = 200
mycondition["th"] = 45

In [140]:
print(fit1.get_Loss(mycondition))

0.126


In [3]:
from enum import Enum
from enum import auto
from Fitting import Fitting
import math

In [5]:
class Duct(object):
    
    class REF(Enum):
        FLOW = auto()
        SIZE = auto()
        LOSS = auto()
        VELOCITY = auto()
    
    FLOW = REF.FLOW
    SIZE = REF.SIZE
    LOSS = REF.LOSS
    VELOCITY = REF.VELOCITY
    
    class DuctType(Enum):
        RECTANGULAR = auto()
        ROUND = auto()
        OVAL = auto()
    
    RECT = DuctType.RECTANGULAR
    ROUND = DuctType.ROUND
    OVAL = DuctType.OVAL
        
    def __init__(self, dtype:DuctType=DuctType.ROUND,
                 diam:float=300.,
                 size:list=[300., 300.],
                 length:float=1.22,
                 flow:float=100.,
                 e:float=0.09,
                 nu:float=1.506e-5,
                 rho:float=1.204):
        self.__name = "duct"
        self.__fittings = []
        self.__dtype = dtype
        try:
            if (len(size) == 2):
                self.__size = size
            else:
                self.__size = [300, 300]
        except TypeError as e:
            self.__size = [diam, 0.]  
        self.__flow = flow
        self.__roughness = e
        self.__viscosity = nu
        self.__density = rho
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, value):
        self.__name = value
    
    def add_fitting(self, fitt:Fitting, cond:dict):
        self.__fittings.append((fitt, cond))
    
    def remove_fitting(self, i:int):
        del(self.__fittings[i])
    
    def clear_fittings(self, i:int):
        self.__fittings = []
    
    def is_width_fixed(self):
        return self.__is_fixed
    
    def fix_width(self, yn:bool = True):
        self.__is_fixed = yn
    
    def __gwi(self) -> int:
        if is_width_fixed():
            return 1
        else:
            return 0
    
    @property
    def dtype(self):
        return self.__dtype
    
    @dtype.setter
    def dtype(self, value):
        try:
            self.__dtype = value
        except:
            print("Bandera")
            self.__dtype = value
    @property
    def flow(self):
        return self.__flow
    
    @flow.setter
    def flow(self, flow:float, ref:REF):
        if (flow < 0 or flow == self.__flow):
            return
        if (ref == LOSS):
            refval = self.loss_rate
            self.flow(flow, REF.SIZE)
            e = 1.0
            count = 0
            rate = 100.
            index = self.__gwi()
            while (e > 0.0001 or count < 1000):
                if (refval < self.loss_rate):
                    self.__size[index] += rate
                    if (refval > self.loss_rate):
                        self.__size[index] -= rate
                        rate /= 10.
                elif (refval > self.loss_rate):
                    self.__size[index] -= rate
                    if (refval < self.loss_rate or self.__size[index] < 0.):
                        a_size[index] += rate;
                        rate /= 10.
                count += 1
                if count == 999:
                    print("Stopped by counter")
                e = abs(self.loss_rate - refval)
        elif (ref == VELOCITY):
            refval = self.fluid_velocity
            self.flow(flow, REF.SIZE)
            e = 1.0
            count = 0
            rate = 100.
            index = self.__gwi()
            while (e > 0.0001 or count < 1000):
                if (refval < self.fluid_velocity):
                    self.__size[index] += rate
                    if (refval > self.fluid_velocity()):
                        self.__size[index] -= rate
                        rate /= 10.
                elif (refval > self.fluid_velocity):
                    self.__size[index] -= rate
                    if (refval < self.fluid_velocity() or self.__size[index] < 0.):
                        a_size[index] += rate
                        rate /= 10.
                count += 1
                if count == 999:
                    print("Stopped by counter")
                e = abs(self.loss_rate - refval)
        else: ## ref == SIZE or ref == FLOW:
            self.__flow = flow
    
    @property
    def size(self):
        return self.__size
    
    @size.setter
    def size(self, value):
        if (len(value) == 2 and value[0] > 0. and value[1] > 0):
            self.__size = value
    
    def set_size(self, size:list = [], diam:float = 300., dtype:DuctType = DuctType.ROUND, ref:REF = FLOW):
        # The reference values are saved before modify the duct
        ref_vel = self.get_fluid_vel()
        ref_loss = self.get_loss_rate()
        if (dtype != self.__dtype):
            print("Type of duct changed")
            self.__dtype = dtype
            if (dtype == ROUND):
                self.__size = [diam, 0.]
            else:
                self.__size = size
        else:
            if (dtype == ROUND):
                self.set_De(diam, ref=REF)
                return
        if (REF == VELOCITY):
            self.__flow = self.get_fluid_area() * ref_vel / 1000.      
        else:
            pass
            ## Pending to define
    
    @property
    def roughness(self):
        return self.__roughness
    
    @roughness.setter
    def roughness(self, value:float):
        if (value > 0.):
            self.__roughness = value
            
    @property
    def density(self):
        return self.__density
    
    @density.setter
    def density(self, value:float):
        if (value > 0. and value < 5.0):
            self.__density = value
    
    @property
    def viscosity(self):
        return self.__viscosity
    
    @viscosity.setter
    def viscosity(self, value:float):
        if (value > 0. and value < 1e-4):
            self.__viscosity = value
    
    @staticmethod
    def get_area(size:list, dtype:DuctType) -> float:
        if (dtype == Duct.RECT):
            return size[0] * size[1] / 1.0e6
        elif (dtype == Duct.ROUND):
            return math.pi * size[0] * size[0] / 4.0e6
        else: ## self.__dtype == OVAL
            return math.pi * size[1] * size[1] / 4.0e6\
              + size[1] * (size[0] - size[1]) / 1.0e6
    
    def get_fluid_area(self):
        return math.pi * self.get_De(self.size, self.dtype) ** 2.0 / 4.0e6
    
    @staticmethod
    def get_perim(size:list, dtype:DuctType) -> float:
        if (dtype == Duct.RECT):
            return 2.0 * (size[0] + size[1])
        elif (dtype == Duct.ROUND):
            return math.pi * size[0]
        else: ## self.__dtype == OVAL
            return math.pi * size[1] + 2 * (size[0] - size[1])
    
    @classmethod
    def get_hid_diam(cls, size:list, dtype:DuctType) -> float:
        if (dtype == Duct.ROUND):
            return size[0]
        else:
            return 4.0e6 * cls.get_area(size, dtype) / cls.get_perim(size, dtype)
        
    @classmethod
    def get_De(cls, size:list, dtype:DuctType):
        if (dtype == Duct.RECT):
            return 1.3 * math.pow(size[0] * size[1], 0.625)\
              / math.pow(size[0] + size[1], 0.25)
        elif (dtype == Duct.ROUND):
            return size[0]
        else: ## dtype == Duct.OVAL
            return 1.55 * math.pow(cls.get_area(size, dtype=dtype), 0.625) / math.pow(cls.get_perim(size, dtype=dtype), 0.25)
    
    def set_De(self, d:float, ref:REF):
        if (d < 0.):
            return
        ref_vel = self.get_fluid_vel()
        ref_loss = self.get_loss_rate()
        adjust_size_by_De(self.size, diam=d, mode="h")
        if (ref == self.REF.VELOCITY):
            self.__flow = ref_vel
            ##Pending to define
        else:
            pass
            ##Pending to define
    
    @staticmethod
    def adjust_size_by_De(size:list, diam:float, side:str ="h", dtype:DuctType = RECT) -> list:
        if dtype == ROUND:
            return size[0]
        if (side == "w"):
            width = size[0]
            return [width, size[1]]
        else:           
            height = size[1]
            return [size[0], height]
        
    @classmethod    
    def get_size_of_De(cls, initial_size:list, De:float, side:str="h", dtype:DuctType = RECT) -> float:
        acc = 1000.
        counter = 0
        while (acc > 0.1 and counter < 1000):
            if (side == "w"):
                width = initial_size[0]
                dw = cls.__get_slope_De([width, initial_size[1]], side = "w", dtype=dtype)
                acc = (cls.get_De([width, initial_size[1]], dtype=dtype) - De) / dw
                width -= acc
                new_size = [width, initial_size[1]]
            else:
                height = initial_size[1]
                dh = cls.__get_slope_De([initial_size[0], height], side = "h", dtype=dtype)
                acc = (cls.get_De([initial_size[0], height], dtype=dtype) - De) / dh
                height -= acc
                new_size = [initial_size[0], height]
            counter += 1
        if (counter == 1000):
            print("Value was not found")
            return initial_size
        else:
            return new_size
        
    @classmethod
    def __get_slope_De(cls, size:list, side:str="h", dtype:DuctType = RECT) -> float:
        delta = 0.00001
        if (side == "w"):
            return (cls.get_De([size[0] + delta, size[1]], dtype=dtype) - cls.get_De(size, dtype=dtype)) / delta
        else:
            return (cls.get_De([size[0], size[1] + delta], dtype=dtype) - cls.get_De(size, dtype=dtype)) / delta
        
    def get_velocity(self):
        return self.__flow / self.get_area(self.size, self.dtype) / 1000.
    
    def get_fluid_vel(self):
        return self.__flow / self.get_fluid_area() / 1000.
    
    def set_fluid_vel(self, vel:float, ref:REF):
        if (v < 0.):
            return
        if (ref == self.DuctType.FLOW):
            pass
            ## Pending to define
        elif (ref == self.DuctType.LOSS):
            pass
            ## Pending to define
        elif (ref == self.DuctType.SIZE):
            pass
            ## Pending to define
        else: ## ref == self.DuctType.VELOCITY
            pass
        
    def get_vel_pressure(self):
        return self.__density / 2. * self.get_fluid_vel() ** 2.
    
    def get_ffactor(self):
        f = self.roughness / self.get_hid_diam(self.size, self.dtype)
        e = 10.
        count = 0
        while (e > 0.0001 and count < 999):
            f = self.FColebrook(f)
            e = abs(f - self.FColebrook(f))
            count += 1
        if (count == 999):
            print("Finished by count")
        return f

    def set_loss_rate(self, loss:float, ref:REF):
        if (loss < 0.0 or loss == self.get_loss()):
            return
        rate = 100.
        e = 100.
        count = 0
        if (ref == self.REF.FLOW):
            while (e > 0.00001 or count < 999):
                if (loss < self.get_loss()):
                    self.set_De(self.get_De() + rate)
                    if (loss > self.get_loss()):
                        self.set_De(self.get_De() - rate)
                        rate /= 10.;
                else:
                    self.set_De(get_De() - rate)
                    if (loss < self.get_loss() or get_De() - rate < 0.):
                        rate /= 10.
                count += 1
                if (count == 999):
                    print("Stopped by counter")
                e = abs(loss - self.get_loss())
        elif (ref == self.REF.VELOCITY):
            pass
            # Pending to define
        elif (rel == self.REF.SIZE):
            while (e > 0.00001):
                if (loss > get_loss()):
                    self.set_flow(self.get_flow() + rate, self.REF.SIZE)
                    if (loss < self.get_loss()):
                        self.set_flow(self.get_flow() + rate, self.REF.SIZE)
                elif (loss < self.get_loss()):
                    self.set_flow(self.get_flow() - rate, self.REF.SIZE)
                    if (loss > self.get_loss() or self.get_flow() - rate < 0.):
                        self.set_flow(self.get_flow() + rate, self.REF.SIZE)
        else:  # rel == self.REF.LOSS
            return
        
    def get_loss_rate(self):
        return 1000. * self.get_ffactor() / self.get_De(self.size, self.dtype) * self.get_vel_pressure()
    
    def get_loss(self, length, output_type:int = 0):
        if (output_type > 2):
            output_type = 0
        Fitloss = 0.0
        for i in range(len(self.__fittings)):
            if (self.__fittings[i][0].fittype == "UNCLASSIFIED" or \
              self.__fittings[i][0].fittype == "TEE" or \
              self.__fittings[i][0].fittype == "WYE"):
                Fitloss += self.__fittings[i][0].get_Loss(self.__fittings[i][1])[output_type]
            else:
                Fitloss += self.__fittings[i][0].get_Loss(self.__fittings[i][1])
        return self.get_loss_rate() * length + Fitloss * self.get_vel_pressure()
    
    def get_Reynolds(self):
        return self.get_hid_diam(self.size, self.dtype) * self.get_velocity() / 1000. / self.__viscosity
    
    def FColebrook(self, f:float):
        return 1. / math.pow(-2.0 * math.log10(self.roughness / 3.7 / self.get_hid_diam(self.size, self.dtype)\
          + 2.51 / self.get_Reynolds() / math.sqrt(f)), 2.0)
    
    def __str__(self):
        result = ""
        result += "Flow (L/s):        {:2f}".format(self.flow) + '\n'
        if (self.dtype == self.DuctType.ROUND):
            result += "Diámetro (mm):     {:2f}".format(self.size[0]) + '\n'
        else:
            result += "Ancho (mm):        {:2f}".format(self.size[0]) + '\n'
            result += "Alto (mm):         {:2f}".format(self.size[1]) + '\n'
        result += "Eq. Diam. (mm):    {:2f}".format(self.get_De(self.size, self.dtype)) + '\n'
        result += "Hid. Diam (mm):    {:2f}".format(self.get_hid_diam(self.size, self.dtype)) + '\n'
        result += "Flow Area (m2):    {:2f}".format(self.get_fluid_area()) + '\n'
        result += "Fluid Vel. (m/s):  {:2f}".format(self.get_fluid_vel()) + '\n'
        result += "Reynolds (-):      {:2f}".format(self.get_Reynolds()) + '\n'
        result += "Friction fac. (-): {:2f}".format(self.get_ffactor()) + '\n'
        result += "Vel. Press. (Pa):  {:2f}".format(self.get_vel_pressure()) + '\n'
        result += "Head loss (Pa/m):  {:2f}".format(self.get_loss_rate())
        return result

In [109]:
Duct.get_size_of_De(initial_size=[300, 300], De=400., side="h")

[300, 6182816.905377347]

In [110]:
duct1 = Duct(size=[300, 6182817], dtype=Duct.RECT)

In [111]:
print(duct1)

Finished by count
Finished by count
Flow (L/s):        100.000000
Ancho (mm):        300.000000
Alto (mm):         6182817.000000
Eq. Diam. (mm):    16174.608056
Hid. Diam (mm):    599.970888
Flow Area (m2):    205.474254
Fluid Vel. (m/s):  0.000487
Reynolds (-):      2.147818
Friction fac. (-): 250.727896
Vel. Press. (Pa):  0.000000
Head loss (Pa/m):  0.000002


In [160]:
duct1.add_fitting(fit1, mycondition)

In [161]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=0)))

Pérdida total (Pa): 0.187586


In [164]:
with open("ASHRAE_SR5-11.json") as file:
    wye = json.load(file)

In [165]:
fit2 = Fitting(wye)

In [166]:
print(fit2)

ASHRAE SR5-11


In [167]:
print(wye)

{'name': 'ASHRAE SR5-11', 'type': 'Tee', 'system': 'Suppy', 'description': 'Rectangular Main to Round Tap, Diverging', 'rel_xo': 'Q_o/Q_i', 'x_o': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_yo': 'A_o/A_i', 'y_o': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_x1': 'Q_1/Q_i', 'x_1': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'rel_y1': 'A_1/A_i', 'y_1': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'data_o': [[0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.98, 0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.48, 0.31, 0.04, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0], [7.55, 0.98, 0.18, 0.04, 0.02, 0.0, 0.0, 0.0, 0.0], [13.18, 2.03, 0.49, 0.13, 0.04, 0.0, 0.01, 0.0, 0.0], [20.38, 3.48, 0.98, 0.31, 0.1, 0.04, 0.02, 0.01, 0.0], [29.15, 5.32, 1.64, 0.6, 0.23, 0.09, 0.04, 0.02, 0.01], [39.48, 7.55, 2.47, 0.98, 0.42, 0.18, 0.08, 0.04, 0.02], [51.37, 10.17, 3.48, 1.46, 0.67, 0.31, 0.15, 0.07, 0.04]], 'data_1': [[1.58, 0.94, 0.83, 0.79, 0.77, 0.76, 0.76, 0.76, 0.75], [4.2, 1.58, 1.1, 0.9

In [168]:
cond2={}
cond2["A_i"] = 200
cond2["A_o"] = 200
cond2["A_1"] = 200
cond2["Q_i"] = 450
cond2["Q_o"] = 300
cond2["Q_1"] = 150

In [169]:
fit2.get_Loss(cond2)

(0.29333333333333333, 8.900000000000002)

In [170]:
duct1.add_fitting(fit2, cond2)

In [171]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=0)))

Pérdida total (Pa): 0.435072


In [172]:
print("Pérdida total (Pa): {:2f}".format(duct1.get_loss(length=1.22, output_type=1)))

Pérdida total (Pa): 7.696521


In [7]:
duct1 = Duct(size=[300, 400], flow=500, dtype=Duct.RECT)

In [8]:
print(duct1)

Flow (L/s):        500.000000
Ancho (mm):        300.000000
Alto (mm):         400.000000
Eq. Diam. (mm):    377.708815
Hid. Diam (mm):    342.857143
Flow Area (m2):    0.112048
Fluid Vel. (m/s):  4.462373
Reynolds (-):      94858.660596
Friction fac. (-): 0.019538
Vel. Press. (Pa):  11.987490
Head loss (Pa/m):  0.620074


In [ ]:
.report()